In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

In [2]:
def missing_data_script(name_of_df): 
    total = name_of_df.isnull().sum().sort_values(ascending=False) 
    percent = (name_of_df.isnull().sum()/name_of_df.isnull().count()).sort_values(ascending=False) 
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) 
    return(missing_data)

In [65]:
df_gigachad = pd.read_csv('beeline_antispam_hakaton_id_samples.csv')

In [66]:
df_train = pd.read_csv('time_key=2021-11-01.csv')

In [67]:
df_gigachad.shape

(46235, 3)

In [68]:
df_ans = pd.read_csv('beeline_antispam_hakaton_id_samples.csv').dropna().drop('split',axis = 1)

df_train =  df_train.merge(df_ans,left_on= 'id_a' ,right_on = 'id').drop(['id', 'start_time_local','time_key'],axis = 1)

In [7]:
import xgboost as xgb

In [8]:
df_train.head()

,id_a,id_b,time_zone,duration,forward,zero_call_flg,source_b,source_f,num_b_length,target
0,67596082,43588153,3,0,0,3,0,1,10,4.0
1,67596082,82184937,3,0,0,5,0,1,10,4.0
2,67596082,48927355,0,0,0,5,0,1,10,4.0
3,67596082,10241386,3,0,0,0,0,1,10,4.0
4,67596082,229405,0,0,1,5,1,0,10,4.0


In [11]:
clf = xgb.XGBClassifier(max_depth=5, objective='multi:softmax', n_estimators=1000, 
                        num_classes=5)

In [69]:
y_train = df_train['target']

In [70]:
x_train = df_train.drop(['target'], axis=1)

In [14]:
x_train.head()

,id_a,id_b,time_zone,duration,forward,zero_call_flg,source_b,source_f,num_b_length
0,67596082,43588153,3,0,0,3,0,1,10
1,67596082,82184937,3,0,0,5,0,1,10
2,67596082,48927355,0,0,0,5,0,1,10
3,67596082,10241386,3,0,0,0,0,1,10
4,67596082,229405,0,0,1,5,1,0,10


In [15]:
y_train.head()

0    4.0
1    4.0
2    4.0
3    4.0
4    4.0
Name: target, dtype: float64

In [37]:
clf.fit(X_train, y_train)

[16:48:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1000,
              n_jobs=0, num_classes=5, num_parallel_tree=1,
              objective='multi:softmax', predictor='auto', random_state=0, ...)

In [16]:
from sklearn.model_selection import train_test_split

In [71]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, 
                                                    train_size=0.67, 
                                                    random_state=42)

In [18]:
y_test

825720     4.0
2315181    4.0
2278142    4.0
882916     4.0
1664989    4.0
          ... 
1430734    4.0
2511916    1.0
2204090    4.0
1782379    4.0
1171125    4.0
Name: target, Length: 924105, dtype: float64

In [38]:
y_pred = clf.predict(X_test)

In [39]:
clf.score(X_test,y_test)

0.9953912163661056

In [19]:
from sklearn.metrics import fbeta_score

In [47]:
fbeta_score(y_pred, y_test,average = 'macro', beta = 0.5)

0.9619194674779369

In [48]:
clf

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1000,
              n_jobs=0, num_classes=5, num_parallel_tree=1,
              objective='multi:softmax', predictor='auto', random_state=0, ...)

In [20]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=0.5)
ftwo_scorer

make_scorer(fbeta_score, beta=0.5)

In [21]:
from sklearn.model_selection import GridSearchCV

In [33]:
parameters = {
    'n_estimators': range(50, 151, 50),
}


estimator = xgb.XGBClassifier(max_depth=5,
    tree_method='gpu_hist',
    gpu_id = -1,
    objective='multi:softmax',
    nthread=4,
    seed=42,
    num_classes=5
)


grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = ftwo_scorer,
    cv = 5,
    verbose=True
)


In [34]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[05:31:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "C:\anaconda\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py", line 1195, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(y_true, y_pred,
  File "C:\anaconda\lib\site-packag

[05:31:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "C:\anaconda\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py", line 1195, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(y_true, y_pred,
  File "C:\anaconda\lib\site-packag

[05:32:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




...

C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "C:\anaconda\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py", line 1195, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(y_true, y_pred,
  File "C:\anaconda\lib\site-packag

[05:34:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "C:\anaconda\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py", line 1195, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(y_true, y_pred,
  File "C:\anaconda\lib\site-packag

[05:34:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=-1, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=5,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
         

In [71]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=0.5)
ftwo_scorer

make_scorer(fbeta_score, beta=0.5)

In [35]:
grid_search.best_params_

{'n_estimators': 50}

In [45]:
clf = xgb.XGBClassifier(max_depth=5, objective='multi:softmax', n_estimators=125, 
                        num_classes=5)

In [46]:
clf.fit(X_train, y_train)

[05:51:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=150,
              n_jobs=0, num_classes=5, num_parallel_tree=1,
              objective='multi:softmax', predictor='auto', random_state=0, ...)

In [72]:
y_pred = clf.predict(X_test)

In [50]:
fbeta_score(y_test, y_pred,average = 'macro', beta = 0.5)

0.9261447473335412

In [78]:
clf = xgb.XGBClassifier(
    max_depth=5, 
    objective='multi:softmax',
    n_estimators=100, 
    num_classes=5)

In [79]:
clf.fit(X_train, y_train)

[06:17:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_classes=5, num_parallel_tree=1,
              objective='multi:softmax', predictor='auto', random_state=0, ...)

In [80]:
y_pred = clf.predict(X_test)

In [81]:
fbeta_score(y_test, y_pred,average = 'macro', beta = 0.5)

0.9040204365866323

In [90]:
clf_GPU = xgb.XGBClassifier(
    tree_method='gpu_hist',
    gpu_id = -1,
    max_depth=5, 
    objective='multi:softmax',
    n_estimators=100, 
    num_classes=5,
    random_state = 42)

In [91]:
clf_GPU.fit(X_train, y_train)

[06:37:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_classes=5, num_parallel_tree=1,
              objective='multi:softmax', predictor='auto', random_state=42, ...)

In [92]:
y_pred = clf_GPU.predict(X_test)

In [93]:
fbeta_score(y_test, y_pred,average = 'macro', beta = 0.5)

0.8071651807715602

In [86]:
clf_GPU_2 = xgb.XGBClassifier(
    tree_method='gpu_hist',
    gpu_id = -1,
    max_depth=5, 
    objective='multi:softmax',
    n_estimators=50, 
    num_classes=5)

In [87]:
clf_GPU_2.fit(X_train, y_train)

[06:32:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_classes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
              num_classes=5, num_parallel_tree=1, objective='multi:softmax',
              predictor='auto', random_state=0, ...)

In [88]:
y_pred = clf_GPU_2.predict(X_test)

In [89]:
fbeta_score(y_test, y_pred,average = 'macro', beta = 0.5)

0.7853381902481299